# Specifications of postprocessing specific subclasses

> This module defines the specifications of postprocessing related subclasses of `ProcessingStrategy` and `ProcessingObject`

In [ ]:
#| default_exp postprocessing/specs

In [ ]:
#| export

from abc import abstractmethod
from typing import Dict, List
from skimage import io

from findmycells.core import ProcessingObject, ProcessingStrategy
from findmycells.database import Database
from findmycells import utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

class PostprocessingStrategy(ProcessingStrategy):
    
    """
    Extending the `ProcssingStrategy` base class for postprocessing as processing subtype.
    """
    
    @property
    def processing_type(self):
        return 'postprocessing' 

In [ ]:
#| export

class PostprocessingObject(ProcessingObject):
    
    """
    Extending the `ProcessingObject` base class for postprocessing as processing subtype.
    """
    
    def __init__(self, database: Database, file_ids: List[str], strategies: List[PostprocessingStrategy], segmentations_to_use: str) -> None:
        super().__init__(database = database, file_ids = file_ids, strategies = strategies)
        self.file_id = file_ids[0]
        self.file_info = self.database.get_file_infos(identifier = self.file_id)
        if segmentations_to_use == 'semantic':
            path = self.database.semantic_segmentations_dir
        elif segmentations_to_use == 'instance':
            path = self.database.instance_segmentations_dir
        else:
            raise ValueError("'segmentations_to_use' has to be either 'semantic' or 'instance'.")
        self.postprocessed_segmentations = utils.load_zstack_as_array_from_single_planes(path = path, file_id = self.file_id)
        self.rois_dict = self.database.area_rois_for_quantification[self.file_id]
        self.segmentations_per_area_roi_id = dict()


    @property
    def processing_type(self):
        return 'postprocessing'
            
    
    def save_postprocessed_segmentations(self) -> None:
        for area_roi_id in self.segmentations_per_area_roi_id.keys():
            for plane_index in range(self.segmentations_per_area_roi_id[area_roi_id].shape[0]):
                image = self.segmentations_per_area_roi_id[area_roi_id][plane_index]
                filepath = self.database.quantified_segmentations_dir.joinpath(area_roi_id)
                if filepath.is_dir() == False:
                    filepath.mkdir()
                filename_path = filepath.joinpath(f'{self.file_id}-{str(plane_index).zfill(3)}_postprocessed_segmentations.png')
                io.imsave(filename_path, image, check_contrast=False)


    def add_processing_specific_infos_to_updates(self, updates: Dict) -> Dict:
        return updates

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()